In [1]:
from tensorflow import keras
import tensorflow as tf
import os
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image
from tensorflow.keras import mixed_precision
import csv
import tensorflow_hub as hub
import dataprocessing


tf.config.list_physical_devices('GPU')

2023-04-30 00:58:29.302170: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 00:58:29.418570: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-30 00:58:29.941615: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/aaaabbee/miniconda3/envs/tf/lib/:/home/aaaabbee/miniconda3/envs/tf/lib/python3.9/site-packages/nvidia/cudnn/lib
2023-04-30 00:58:29.943708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] C

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
def generate_augmented_images(batch_size,
                              img_size,
                              normalize=True,
                              data_format="channels_last",
                              train_location="dataset/train_images/",
                              valsplit=True):

    if (normalize == True):
        aug_gens = ImageDataGenerator(
            rescale=1.0 / 255,
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            validation_split=0.1,
            rotation_range=10,
            shear_range=0.15,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=False,
            vertical_flip=False,
            data_format=data_format,
            brightness_range=(0.9, 1.1),
            fill_mode="constant",
        )
    else:
        aug_gens = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            validation_split=0.1,
            rotation_range=10,
            shear_range=0.15,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=False,
            vertical_flip=False,
            data_format=data_format,
            brightness_range=(0.9, 1.1),
            fill_mode="constant",
        )
    if (valsplit):

        train_data = aug_gens.flow_from_directory(train_location,
                                                  subset="training",
                                                  seed=1447,
                                                  target_size=(img_size,
                                                               img_size),
                                                  batch_size=batch_size,
                                                  class_mode="categorical")

        val_data = aug_gens.flow_from_directory(train_location,
                                                subset="validation",
                                                seed=1447,
                                                target_size=(img_size,
                                                             img_size),
                                                batch_size=batch_size,
                                                class_mode="categorical")
    else:
        train_data = aug_gens.flow_from_directory(train_location,
                                                  subset=None,
                                                  seed=1447,
                                                  target_size=(img_size,
                                                               img_size),
                                                  batch_size=batch_size,
                                                  class_mode="categorical")

        val_data = None

    return train_data, val_data


def generate_nonaugmented_images(batch_size,
                                 img_size,
                                 normalize=True,
                                 train_location="dataset/train_images/",
                                 valsplit=True,
                                 class_mode="categorical"):

    if (normalize == True):
        aug_gens = ImageDataGenerator(
            rescale=1.0 / 255,
            validation_split=0.1,
        )
    else:
        aug_gens = ImageDataGenerator(validation_split=0.1)
    if (valsplit):
        train_data = aug_gens.flow_from_directory(train_location,
                                                  subset="training",
                                                  seed=1447,
                                                  target_size=(img_size,
                                                               img_size),
                                                  batch_size=batch_size,
                                                  class_mode=class_mode)

        val_data = aug_gens.flow_from_directory(train_location,
                                                subset="validation",
                                                seed=1447,
                                                target_size=(img_size,
                                                             img_size),
                                                batch_size=batch_size,
                                                class_mode=class_mode)
    else:
        train_data = aug_gens.flow_from_directory(train_location,
                                                  subset=None,
                                                  seed=1447,
                                                  target_size=(img_size,
                                                               img_size),
                                                  batch_size=batch_size,
                                                  class_mode=class_mode)
        val_data = None
    return train_data, val_data


def generate_test_labels(test_location="dataset/test_images"):

    img_id = []
    for fileName in os.listdir(test_location):
        img_id.append(fileName)
    return img_id


def tta_prediction(model,
                   batch_size,
                   img_size,
                   normalize=True,
                   data_format='channels_last',
                   test_location="dataset/test_images"):

    if (normalize == True):
        aug_gens = ImageDataGenerator(
            rescale=1.0 / 255,
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            validation_split=0.1,
            rotation_range=10,
            shear_range=0.25,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=False,
            vertical_flip=False,
            data_format=data_format,
            fill_mode="constant",
        )
    else:
        aug_gens = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            validation_split=0.1,
            rotation_range=10,
            shear_range=0.25,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=False,
            vertical_flip=False,
            data_format=data_format,
            fill_mode="constant",
        )

    tta_steps = 10
    predictions = []
    for i in tqdm(range(tta_steps)):
        preds = model.predict(aug_gens.flow_from_directory(
            directory=test_location,
            target_size=(img_size, img_size),
            batch_size=batch_size,
            classes=['.'],
            shuffle=False,
        ),
                              workers=16)
        predictions.append(preds)

    final_pred = np.mean(predictions, axis=0)
    return final_pred


# This does not work
def get_data(batch_size, img_size):
    traindf = pd.read_csv("otherdataset/train/_annotations.csv", dtype=str)
    valdf = pd.read_csv("otherdataset/valid/_annotations.csv", dtype=str)
    testdf = pd.read_csv("otherdataset/test/_annotations.csv", dtype=str)

    aug_gens = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        shear_range=0.25,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False,
        data_format='channels_first',
        fill_mode='nearest',
    )

    train = aug_gens.flow_from_dataframe(dataframe=traindf,
                                         directory="otherdataset/train/",
                                         x_col='filename',
                                         y_col='class',
                                         batch_size=batch_size,
                                         seed=1447,
                                         class_mode='categorical',
                                         target_size=(img_size, img_size))

    valid = aug_gens.flow_from_dataframe(dataframe=valdf,
                                         directory="otherdataset/valid/",
                                         x_col='filename',
                                         y_col='class',
                                         batch_size=batch_size,
                                         seed=1447,
                                         class_mode='categorical',
                                         target_size=(img_size, img_size))

    test = aug_gens.flow_from_dataframe(dataframe=testdf,
                                        directory="otherdataset/test/",
                                        x_col='filename',
                                        y_col='class',
                                        batch_size=batch_size,
                                        seed=1447,
                                        class_mode='categorical',
                                        target_size=(img_size, img_size))

    return train, valid, test

In [3]:
label_dict = {}
for i, line in enumerate(open("wnids.txt", "r")):
    label_dict[line.rstrip("\n")] = int(i)

label_dict

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 '_': 26}

In [4]:
batch_size = 64
img_size = 224
num_classes = 26

In [5]:
train_data, val_data = generate_augmented_images(
        batch_size,
        img_size,
        normalize=False,
        train_location="otherdataset/train_images")

Found 1503 images belonging to 26 classes.
Found 153 images belonging to 26 classes.


In [6]:
test_labels = dataprocessing.generate_test_labels(
        test_location="otherdataset/test_images")
test_int = [label_dict[x[0]] for x in test_labels]
test_int

[0,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 8,
 8,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 12,
 12,
 12,
 13,
 13,
 13,
 14,
 14,
 14,
 15,
 16,
 16,
 17,
 17,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 24,
 24,
 25,
 25,
 25,
 25]

### Transfer Learning

In [7]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor_model = mobilenet_v2

feature_extractor_layer = hub.KerasLayer(feature_extractor_model,
                                             input_shape=(img_size, img_size,
                                                          3),
                                             trainable=True)

2023-04-30 00:58:31.996604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 00:58:31.998544: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-30 00:58:31.999084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-30 00:58:31.999434: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

### Model

In [8]:
### Optimized Neural Network
model = keras.models.Sequential()

# Model Layers
model.add(keras.layers.Rescaling((1. / 255)))
model.add(feature_extractor_layer)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='swish'))
model.add(keras.layers.Dense(256, activation='swish'))
model.add(keras.layers.Dense(num_classes, activation='softmax'))

model.build(input_shape=(None, img_size, img_size, 3))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 256)               262400    
                                                                 
 dense_2 (Dense)             (None, 26)                6682      
                                                                 
Total params: 3,838,810
Trainable params: 3,804,698
Non-

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

num_epochs = 100
lr_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                     patience=4,
                                                     verbose=1,
                                                     factor=0.4,
                                                     min_lr=0.0001)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0.00001,
                                               patience=8,
                                               mode='auto',
                                               restore_best_weights=True)

history = model.fit(train_data,
                        workers=16,
                        epochs=num_epochs,
                        validation_data=val_data,
                        batch_size=batch_size,
                        verbose=1,
                        callbacks=[early_stop, lr_reduction],
                        max_queue_size=30)

Epoch 1/100


2023-04-30 00:58:44.319012: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2023-04-30 00:58:45.207415: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


24/24 [==============================] - 22s 493ms/step - loss: 1.7672 - accuracy: 0.5170 - val_loss: 5.8055 - val_accuracy: 0.2484 - lr: 0.0010
Epoch 2/100
24/24 [==============================] - 12s 376ms/step - loss: 0.5575 - accuracy: 0.8510 - val_loss: 9.9915 - val_accuracy: 0.1634 - lr: 0.0010
Epoch 3/100
24/24 [==============================] - 13s 367ms/step - loss: 0.4240 - accuracy: 0.8902 - val_loss: 10.7637 - val_accuracy: 0.2745 - lr: 0.0010
Epoch 4/100
24/24 [==============================] - 14s 394ms/step - loss: 0.3860 - accuracy: 0.9069 - val_loss: 3.7211 - val_accuracy: 0.4248 - lr: 0.0010
Epoch 5/100
24/24 [==============================] - 13s 367ms/step - loss: 0.3184 - accuracy: 0.9301 - val_loss: 4.8460 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 6/100
24/24 [==============================] - 14s 402ms/step - loss: 0.1984 - accuracy: 0.9681 - val_loss: 3.1012 - val_accuracy: 0.6078 - lr: 0.0010
Epoch 7/100
24/24 [==============================] - 14s 400ms/step -

Final validation accuracy of 96% good enough.

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('asl.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp74_ahuhw/assets


INFO:tensorflow:Assets written to: /tmp/tmp74_ahuhw/assets
2023-04-30 01:12:53.716862: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-04-30 01:12:53.716907: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-04-30 01:12:53.717594: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp74_ahuhw
2023-04-30 01:12:53.730292: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-30 01:12:53.730321: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp74_ahuhw
2023-04-30 01:12:53.772520: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-04-30 01:12:53.789376: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-04-30 01:12:54.531666: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio